<a href="https://colab.research.google.com/github/Antonio-Villarreal/Deep-Learning-For-Dummies/blob/main/VGG16_FMNST_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import cv2
from keras.applications.vgg16 import VGG16
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dense
from keras.models import Model

import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Resize the images
x_train = np.array([cv2.resize(img, (32, 32)) for img in x_train])
x_test = np.array([cv2.resize(img, (32, 32)) for img in x_test])

# Convert the grayscale images to RGB by repeating the same image three times
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)

# Preprocess the data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [40]:
# Create an instance of the VGG16 model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
print(vgg16.summary())

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [41]:
# Create the input layer for the new model
input = Input(shape=(32, 32, 3), name = 'image_input')

In [42]:
# Use the generated model 
output_vgg16_conv = vgg16(input)

In [43]:
# Add a new fully connected layer for classification
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(10, activation='softmax', name='predictions')(x)

In [44]:
# Create the new model
model = Model(input, x)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [45]:
# Train the model
model.fit(x_train, y_train, epochs=1, batch_size=32, validation_data=(x_test, y_test))

1875/1875 [==============================] - 3493s 2s/step - loss: 0.6686 - accuracy: 0.7706 - val_loss: 0.4238 - val_accuracy: 0.8489


In [46]:
# Evaluate the model
score = model.evaluate(x_test, y_test)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

313/313 [==============================] - 76s 243ms/step - loss: 0.4238 - accuracy: 0.8489
Test loss:  0.4237941801548004
Test accuracy:  0.8489000201225281


In [51]:
score = model.evaluate(x_test, y_test)
score

313/313 [==============================] - 76s 244ms/step - loss: 0.4238 - accuracy: 0.8489


[0.4237941801548004, 0.8489000201225281]